# Задача №30: Метод построения HG-LBP дескриптора на основе гистограмм градиентов для детектирования пешеходов.

Предлагается разработать новый дескриптор, обобщающий LBP дескриптор на основе гистограмм модулей градиентов, имеющий свойства композиции HOG-LBP для задачи детектирования пешеходов на изображении. В качестве анализа качества нового дескриптора предлагается использовать графики ошибок детектирования FAR/FRR на базе INRIA.

http://www.machinelearning.ru/wiki/index.php?title=Автоматизация_научных_исследований_в_машинном_обучении_%28практика%2C_В.В._Стрижов%29#.D0.97.D0.B0.D0.B4.D0.B0.D1.87.D0.B0_30

__Данные:__ База данных пешеходов INRIA: http://pascal.inrialpes.fr/data/human/

__Базовой алгоритм:__ Xiaoyu Wang, Tony X. Han, Shuicheng Yan. An HOG-LBP Human Detector with Partial Occlusion Handling \\ ICCV 2009

__Решение:__ Одним из вариантов обобщения LBP может быть использование вместо гистограмм распределения точек по LBP-коду, гистограмм распределения модулей градиентов точек в блоке по LBP-коду (HG-LBP). Предлагается для основы экспериментов использовать библиотеку OpenCV, в которой реализованы алгоритмы HOG и LBP. Необходимо модифицировать исходный код реализации LBP и вставить подсчет модулей градиента и накопление соответствующей гистограммы по LBP. Необходимо написать программу чтения базы INRIA, обучения по ней метода линейного SVM на исходных и модифицированных дескрипторах, сбора статистики детектирования и построения DET-графиков FAR/FRR.

Для работы необходимо установить OpenCV для Python
1. Запустить __Anaconda Promt__ от имени администратора
2. Ввести __conda install -c conda-forge opencv__

In [1]:
import cv2
import numpy as np

In [2]:
# далее задаются параметры HOG-a, они выставлены как в статье.
win_size = (64,128) 
block_size = (16,16)
block_stride = (8,8)
cell_size = (8,8)
nbins = 9
deriv_aperture = 1
win_sigma = -1
histogram_norm_type = 0
l2_hys_threshold = 2.0000000000000001e-01
gamma_correction = 0
nlevels = 64
hog = cv2.HOGDescriptor(win_size,block_size,block_stride,cell_size,nbins,deriv_aperture,win_sigma,
                        histogram_norm_type,l2_hys_threshold,gamma_correction,nlevels)


In [3]:
win_stride = (8,8) #шаг скользящего окна в пикселях по ширине и высоте
path_to_image = "INRIAPerson/Train/pos/crop_000010.png"
img = cv2.imread(path_to_image)
descriptor = hog.compute(img, win_stride)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

В базе INRIA размер изображений содержащих пешеходов  - 96х160(train) и 70x134(test). Человек расположен по центру.

Вам нужно получить вектор признаков(дескриптор) для окна 64х128. По умолчанию compute() строит дескрипторы для всех окон размером 64х128 методом скользящего окна, с шагом win_stride по обеим осям. Но если вам нужно получить дескриптор только для одного окна 64х128, то у метода есть аргумент locations, в котором можно передать координаты верхнего левого угла интересующего вас окна (либо список таких координат)

In [4]:
img = cv2.imread(path_to_image)
height, width = img.shape[:2]
locations = [((width-64)//2, (height-128)//2)]
descriptor = hog.compute(img, locations=locations)

Проделаем то же в цикле для всех изображений

In [5]:
from os import listdir
from os.path import isfile, join

path_to_pos_images = "INRIAPerson/Train/pos/"
onlyfiles = [f for f in listdir(path_to_pos_images) if isfile(join(path_to_pos_images, f))]
descriptors_pos = list()

Сделаем это в цикле для каждого изображения в папке pos/


In [6]:
#Так как все изображения в тесте одинаковые по размеру, то
# посчитаем константы вне цикла
img = cv2.imread(path_to_pos_images+onlyfiles[0])
height, width = img.shape[:2]
locations = [((width-64)//2, (height-128)//2)]

for im in onlyfiles:
    img = cv2.imread(path_to_pos_images+im)
    descriptors_pos.append(np.squeeze(hog.compute(img, locations=locations)))
    
descriptors_pos = np.stack(descriptors_pos)

Изображения фона(отрицательные примеры)  - изображения произвольного размера. В обучающей выборке их 1218. Авторы HOG для обучения выбирали рандомно 10 окон 64х128 из каждого изображения  - всего 12180 примеров, не содержащих пешеходов.

Для этого, например, можно посчитать дескрипторы по всему изображению с win_stride=(4,4) и потом рандомно выбрать 10 из них.
Сделаем это так же в цикле для всех файлов из папки neg/

In [7]:
path_to_neg_images = "INRIAPerson/Train/neg/"
onlyfiles = [f for f in listdir(path_to_neg_images) if isfile(join(path_to_neg_images, f))]
descriptors_neg = list()

win_stride = (4,4)

for im in onlyfiles:
    img  = cv2.imread(path_to_neg_images+im)
    #получаем дескрипторы изображения и приводим их к рамеру
    #(кол-во окон на изображении)х(рамер дескриптора одного окна)
    #рамер дексриптора для параметров HOG как в статье - 3780
    descriptor = hog.compute(img, win_stride).reshape(-1,3780)
    indexes = np.random.randint(descriptor.shape[0], size=2)
    ten_random_samples = descriptor[indexes]
    descriptors_neg.extend(ten_random_samples)
np.squeeze(descriptors_neg)
descriptors_neg = np.stack(descriptors_neg)

Подготовим данные для обучения

In [8]:
#Создадим маркеры, отвечающие за принадлежность к pos и neg папкам. Пусть pos - 1, а neg - 0. Тогда
labels_pos = [1]*np.array(descriptors_pos).shape[0]
labels_neg = [0]*np.array(descriptors_neg).shape[0]
labels = np.array(labels_pos+labels_neg)

features = descriptors_pos
for i in descriptors_neg:
    features = np.vstack([features, i])

### SVM
Существует два способа дальнейшего развития событий. Мы можем использовать алгоритм svm из sklearn или из opencv
Основной проблемой на данном этапе является правильное задание параметров

#### 1. Обучим SVM от sklearn

In [10]:
from sklearn.svm import SVC
clf = SVC(C=10000,kernel="linear",gamma=0.000001)
clf.fit(features,labels)

SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1e-06, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [57]:
win_stride = (8,8) #шаг скользящего окна в пикселях по ширине и высоте
img = cv2.imread("INRIAPerson/Train/pos/crop_000010.png")
cv2.resize(img, (64,128), interpolation = cv2.INTER_AREA)
height, width = img.shape[:2]
locations = []
for i in range(width//8):
    locations.append((i*8, (height-128)//2))
#locations = [((width-64)//2, (height-128)//2), (width-64, height-128)]
lst_test = hog.compute(img, locations = locations)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(lst_test.shape)

(279720, 1)


In [68]:
print(height//128)
print(width//64)

5
9


In [58]:
lst_test = lst_test.reshape(3780,lst_test.shape[0]//3780)

In [59]:
test_data = np.stack(lst_test)
test_data = test_data.T

In [60]:
pred = clf.predict(test_data)

In [61]:
print(pred)

[1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1
 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1]


In [62]:
for i in range(len(locations)):
    if (pred[i] == 1):
        img = cv2.rectangle(img,locations[i],(locations[i][0]+64,locations[i][1]+128),(255,255,0),3)
        
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
img = cv2.rectangle(img,locations[0],(locations[0][0]+64,locations[0][1]+128),(255,255,0),3)
img = cv2.rectangle(img,locations[1],(locations[1][0]+64,locations[1][1]+128),(255,255,0),3)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 2. SVM от OpenCV

In [74]:
# Set up SVM for OpenCV 3
svm = cv2.ml.SVM_create()
# Set SVM type
svm.setType(cv2.ml.SVM_C_SVC)
# Set SVM Kernel to Radial Basis Function (RBF) 
svm.setKernel(cv2.ml.SVM_RBF)
# Set parameter C
C = 12.5
svm.setC(C)
# Set parameter Gamma
gamma = 0.50625
svm.setGamma(gamma)
 
# Train SVM on training data  
svm.train(features, cv2.ml.ROW_SAMPLE, labels)
 
# Save trained model 
svm.save("digits_svm_model.yml");

Запишем данные в файлы, чтобы позже работать с ними без детектирования

In [53]:
f_1 = open('pos_detection.txt', 'w')
f_2 = open('neg_detection.txt', 'w')

for item in descriptors_pos:
    for i in item:
        f_1.write(str(i[0])+ " ")
    f_1.write('\n')
for item in descriptors_neg:
    for i in item:
        f_2.write(str(i)+ " ")
    f_2.write('\n')
    
f_1.close()
f_2.close()

### TO DO
SVM классификатор
Сначала задаём ему стандартные параметры, а потом используем метод detect()

1. Как он работает? Что он делает?
2. Какую роль играют те детекторы, которые мы обучали? Возможно, его можно/нужно обучить
3. Прочитать другие статьи и, возможно, сделать по-другому
4. Что он выдаёт. Если это координаты в 2D пространстве и его уверенность, то круть. Можно нарисовать график. Но тогда почему 2 признака?


Подготовим данные, чтобы с ними было удобно работать.
Встаёт ещё вопрос, какой SVM брать: из opencv или sklearn

In [69]:
hog.setSVMDetector(hog.getDefaultPeopleDetector())

In [78]:
img = cv2.imread("INRIAPerson/Test/pos/crop001682.png")
a = hog.detectMultiScale(img)

for i in range(len(a[0])):
    img = cv2.rectangle(img,(a[0][i][0],a[0][i][1]) ,(a[0][i][2],a[0][i][3]),(255,255,0),3)
    
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Предполангется, что он возвращает вершины прямоугольника. Но по картинке это как-то не особо заметно. Или он плохо детектирует
В общем, остются вопросы по поводу
1. Как обучать его. Что он принимает? (Возможно, есть в курсах яндекса, если это svm от sklearn)
2. Что он выдаёт? Ебала какая-то. Или он плохо настроен

Протестируем

In [48]:
win_stride = (8,8) #шаг скользящего окна в пикселях по ширине и высоте
path_to_image = "INRIAPerson/Test/pos/crop_000001.png"
img = cv2.imread(path_to_image)
descriptor_ = hog.compute(img, win_stride)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
img = cv2.imread("INRIAPerson/Test/pos/crop_000001.png")
height, width = img.shape[:2]
locations = [((width-64)//2, (height-128)//2)]
descriptor_ = hog.compute(img, locations=locations)
print(descriptor_.shape)

(3780, 1)


In [72]:
# Test on a held out test set
testResponse = svm.predict(descriptor_)[1].ravel()

NameError: name 'svm' is not defined